want to try and optimize feature selection

from what i can see, accident, vehicle, and person datasets seem the most relevant, might try to take some features from there

converting each csv into a dataframe for better handling
- df is our base df, will be writing it to a new csv when done
- finding intersection btwn all three csvs, using that to drop features from df1 and df2.
- i then will be adding df1 and df2 (with only unique cols) into df
- from here, determine features with strongest correlation to target since our max is 20 features
- going to run decision tree model from demo.ipynb on new csv to see if any improvements in accuracy

In [31]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
RANDOM_STATE = 42
files = ["accident23", "person23", "vehicle23"]

df = pd.read_csv("C:\\Users\\15013\\Desktop\\CRSS2023CSV\\accident23.csv")
df1 = pd.read_csv("C:\\Users\\15013\\Desktop\\CRSS2023CSV\\vehicle23.csv")
df2 = pd.read_csv("C:\\Users\\15013\\Desktop\\CRSS2023CSV\\person23.csv")

df3 = pd.DataFrame()

C:\Users\15013\AppData\Local\Temp\ipykernel_18008\1782660601.py:10: DtypeWarning: Columns (57,59,69,73) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("C:\\Users\\15013\\Desktop\\CRSS2023CSV\\vehicle23.csv")


In [32]:
accidents_col = list(df.columns)
vehicles_col = list(df1.columns)
person_col = list(df2.columns)

intersect = []

for x in accidents_col: #there has to be a better way to do this.. 
    if x in vehicles_col:
        if x in person_col:
            intersect.append(x) #if feature is in all 3 dfs

intersect.remove("CASENUM") #we wanna keep casenum since this is our joining feature
#gonna use this list to drop features from df1 and df2

df1 = df1.drop(columns = intersect, errors = "ignore")
df2 = df2.drop(columns = intersect, errors = "ignore")

print(len(df))
print(len(df1))
print(len(df2)) #each file has a differnt num of entries, we need to match case numbers

50103
87461
122388


In [33]:
#merge all three dfs based on casenum
df3 = (
    df
    .merge(df1, on="CASENUM", how="left")
    .merge(df2, on="CASENUM", how="left")
)

df3 = df3[df3['MAX_SEV'] <= 4] #drop where sev vals > 4


#so by this point what we have is df3, which is a dataframe consisting of
#each unique column in the three csvs

In [ ]:
#checking for missing/unknown values 
#most of the code taken from demo.ipynb

#tentative colnames from each csv
acc_cols = [
    "MONTH",
    "DAY_WEEK",
    "HOUR",
    "WEATHER",
    "LGT_COND",
    "HARM_EV", #first harmful event that took place
    "MAN_COLL", #the kind of collision
    "REL_ROAD",
    "TYP_INT",
    "URBANICITY" #was the area urban or rural
]

veh_cols = [
    "VSPD_LIM", #speed limit of the area
    "VSURCOND",
    "ACC_TYPE", #type of accident
    "SPEEDREL", #was accident speeding related
    "DEFORMED" #extent of damage
]

person_cols = [
    "AGE",
    "SEX",
    "PER_TYPE", #role of the person at time of crash i.e. driver, occupant, etc
    "SEAT_POS", #where in car person was sitting
    "REST_USE" #restraint system used i.e. seatbelt, motorcycle helmet, etc
]
df_clean = df3[[acc_cols + veh_cols + person_col]]

UNKNOWN_MAP = {
    #accident csv
    "DAY_WEEK": {9},
    "HOUR": {99},
    "HARM_EV": {99},
    "MAN_COLL": {99, 98},
    "TYP_INT": {99, 98},
    "REL_ROAD": {99, 98},
    "LGT_COND": {9, 8},
    "WEATHER": {98, 99},
    
    #vehicle csv
    "VSPD_LIM": {98, 99},
    "VSURCOND": {99},
    "ACC_TYPE": {5, 10, 16, 33, 43, 49, 53, 63, 67, 75, 85, 91, 99},
    "SPEEDREL": {8, 9},
    "DEFORMED": {8,9},
    
    #person csv
    "AGE": {998, 999},
    "SEX": {8, 9},
    "PER_TYPE": {9,  19},
    "SEAT_POS": {98, 99},
    "REST_USE": {98, 99}
}



rows = []
n = len(df3)
for col in df3.columns:
    codes = UNKNOWN_MAP.get(col, set())
    if len(codes) == 0:
        unk_count = 0
    else:
        unk_count = df3[col].isin(codes).sum()
    rows.append({
        "column": col,
        "unknown_codes": sorted(codes),
        "unknown_count": int(unk_count),
        "unknown_percent": round(100 * unk_count / n, 3) if n else 0.0
    })
unknown_summary = pd.DataFrame(rows).sort_values("unknown_percent", ascending=False)
print(unknown_summary)
df_clean = df3.copy()
for col, codes in UNKNOWN_MAP.items():
    if codes:
        df_clean.loc[df_clean[col].isin(codes), col] = np.nan

In [ ]:
#starting on the model
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score

X = df_clean.drop(columns=['MAX_SEV'])
y = df_clean['MAX_SEV']
for col in X.columns:
    X[col] = X[col].astype('category').cat.codes
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

dt = DecisionTreeClassifier(max_depth=10, min_samples_split= 200, random_state=RANDOM_STATE)
dt.fit(X_train, y_train)

y_pred = dt.predict(X_test)

print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))